In [7]:
import numpy as np
import pandas as pd
from dbfread import DBF

import os
from keras.layers import (
    Input,
    Dense,
    LSTM,
    AveragePooling1D,
    TimeDistributed,
    Flatten,
    Bidirectional,
    Dropout
)
from keras.models import Model  
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_delta=0.01)

In [8]:
BATCH_SIZE = 1024
N_EPOCHS = 30


In [1]:
N_FEATURES = 22

# taken from http://simaaron.github.io/Estimating-rainfall-from-weather-radar-readings-using-recurrent-neural-networks/
THRESHOLD = 73 


In [5]:
df = DBF('./데이터\\B422. 서울시 누적강수량(월평균) 공간데이터\\2. 파일데이터\\2019\\TL_RFL_AGE_2019.dbf',encoding='euc-kr', load=True)
dbf = pd.DataFrame(df)
dbf.head()

,M01,M02,M03,M04,M05,M06,M07,M08,M09,M10,...,ADDR_KOR,DGE_X,DGE_Y,DAMDANG,CONTENT,MK_YEAR,ETC,MTHD_NM,CMDT_NM,SOURCETHM
0,1.0,33.0,19.5,28.0,39.0,48.5,1.96,1.02,26.0,20.0,...,중구 삼일대로 231(예장동 산 5-85),"N 37°33' 21.13""","E 126°59' 31.57""",서울시,풍향/풍속/기온/강수량/강수유무/습도/일사/일조,2009-01-08,서울시청 남산별관(4층),199302.289,550710.248,07(일누적강수량)
1,0.5,27.5,18.0,39.5,32.0,47.5,1.97,1.00,25.0,19.0,...,동대문구 한천로 326(휘경동 348),"N 37°35' 20.58""","E 127°04' 11.28""",서울시,풍향/풍속/기온/강수량/강수유무/습도,2009-01-09,휘경 빗물 펌프장(2층),206164.021,554394.772,07(일누적강수량)
2,0.0,1.0,20.0,32.5,37.5,55.5,1.95,1.14,33.5,20.5,...,성북구 종암로 98-8(종암동주민센터),"N 37°35' 56.62""","E 127°02' 06.73""",서울시,풍향/풍속/기온/강수량/강수유무/습도,2009-01-08,종암동주민센터(5층)//3.15이설완료//,203108.332,555504.066,07(일누적강수량)
3,0.0,46.5,22.0,37.5,33.5,57.0,2.16,1.15,18.0,22.5,...,도봉구 해등로 25(창동 362-1),"N 37°40' 07.93""","E 127°02' 35.15""",서울시,풍향/풍속/기온/강수량/강수유무/습도,2009-01-07,도봉소방서(창동 119안전센터)(2층),203801.837,563251.524,07(일누적강수량)
4,0.5,31.5,14.5,19.5,36.0,44.0,1.82,1.16,25.0,19.0,...,마포구 성중길 82(성산동 445),"N 37°34' 25.43""","E 126°54' 02.53""",서울시,풍향/풍속/기온/강수량/강수유무/습도,2009-01-09,성산빗물펌프장(2층),191229.289,552697.015,07(일누적강수량)


In [ ]:
train_df[train_df.columns[1:]] = train_df[train_df.columns[1:]].astype(np.float32)
